# Podcast Micro-Categories
### Exploratory Notebook
### Unsupervised LDA

In [65]:
from __future__ import print_function
from __future__ import division

import gc
import os
import time

import numpy as np
import pandas as pd
import scipy

import lda

import re

from sklearn.externals import joblib


In [67]:
samp = False
samp = '_samp' if samp else ''

# I. Load

### A. Load Files

In [ ]:
# Define scenario number (s[1-6])
scenario = 's4'

In [137]:
# 1. Load TDM
loader = np.load('../interim/' + scenario + '_tdm' + samp + '.npz')
tdm = scipy.sparse.csr_matrix((loader['data'], loader['indices'], loader['indptr']),
                      shape=loader['shape'])

# 2. Load feature names
feature_names = np.array(pd.read_pickle('../interim/' + scenario + '_names' + samp + '.p'))

# 3. Load show names and subgenres that still remain
shows_concat = pd.read_pickle('../interim/' + scenario + '_shows_concat' + samp + '.p')

# 4. Load full show table so that summaries can be pulled in
shows_full = pd.read_pickle('../interim/pods' + samp + '.p')

# 5. Print shapes and check assertions
print("Episode Term Document Matrix Shape: {:,} shows x  {:,} features".format(tdm.shape[0], tdm.shape[1]))
print("Episode Table Shape: {:,} shows x  {:,} columns (show name and concatenated description)".format(shows_concat.shape[0], shows_concat.shape[1]))
assert tdm.shape[0] == shows_concat.shape[0]
assert len(feature_names) == tdm.shape[1]

Episode Term Document Matrix Shape: 11,795 shows x  363,136 features
Episode Table Shape: 11,795 shows x  2 columns (show name and concatenated description)


### B. Remove duplicates 

In [69]:
# Remove duplicates from full show list (need to handle these earlier on in other scripts)
dupes = shows_full.groupby(['podcast_name', 'subgenre']).filter(lambda group: len(group) > 1).sort('podcast_name')
print("Number of dupes removed: ", dupes.shape)
shows_full = shows_full.drop_duplicates(['podcast_name', 'subgenre'])
print("Number of shows remaining: ", shows_full.shape)

Number of dupes removed:  (151, 18)
Number of shows remaining:  (12319, 18)


### C. Join features from full show table to de-duplicated shows

In [70]:
# Join data from full show table to list of shows in the model

print(shows_full.shape)
print(shows_concat.shape)

shows = pd.merge(shows_concat, shows_full[['podcast_name', 'subgenre', 'show_desc']], 
                 on = ['podcast_name', 'subgenre'], how='left', sort=False)
print(shows.shape)
assert shows.shape[0] == shows_concat.shape[0]
shows.head()

(12319, 18)
(11795, 2)
(11795, 3)


,podcast_name,subgenre,show_desc
0,"! Football Ringtones, Text Tones, Mail Alerts ...",College & High School,Get FREE RINGTONES when you subscribe! Footbal...
1,! World's Funniest Ringtones for iPhone & iPad...,Gadgets,Get FREE RINGTONES when you subscribe to The W...
2,! iPhone Ringtones,Video Games,Get FREE RINGTONES when you subscribe! The Wor...
3,""" Talk to Tony - Radio Show "" Licensed Psychol...",Social Sciences,Dr. Tony Ferretti offers a psychological persp...
4,"""A Tale of Two Cities"" Audiobook (Audio book)",Literature,"""A Tale of Two Cities"" was first published in ..."


# II. Filter

### A. Remove shows with no words in the vocab

In [132]:
# Identify show that have no words in the vocabulary
row_sums = np.array(tdm.sum(axis=1)).flatten() # sum tdm counts across rows (across shows)

print("Number of shows with no words in the vocabulary: ", 
      shows.ix[np.where(row_sums == 0)[0],:].shape[0]) 

Number of shows with no words in the vocabulary:  0


In [74]:
# Remove shows with no words in the vocabulary, from both the original shows table
# and the TDM
shows = shows.ix[np.where(row_sums > 0)[0],:]
tdm = tdm[np.where(row_sums > 0)[0],:]
print(shows.shape)
print(tdm.shape)
assert shows.shape[0] == tdm.shape[0]

(11738, 3)
(11738, 363136)


# III. Train LDA Model

In [75]:
# Create model
model = lda.LDA(n_topics=56, n_iter=75, random_state=42)

# Fit model (takes about 20 minutes for 75 iterations
%time model.fit(tdm)

Wall time: 21min 44s


<lda.lda.LDA instance at 0x000000001CDAFB88>

In [76]:
# (Save/Load model pickle object as needed)
# Pickle model
joblib.dump(model, '../interim/trained_models/lda/lda_' + scenario + samp + '.pkl')


['../interim/trained_models/lda/lda_s4.pkl',
 '../interim/trained_models/lda/lda_s4.pkl_01.npy',
 '../interim/trained_models/lda/lda_s4.pkl_02.npy',
 '../interim/trained_models/lda/lda_s4.pkl_03.npy',
 '../interim/trained_models/lda/lda_s4.pkl_04.npy',
 '../interim/trained_models/lda/lda_s4.pkl_05.npy',
 '../interim/trained_models/lda/lda_s4.pkl_06.npy',
 '../interim/trained_models/lda/lda_s4.pkl_07.npy']

# IV. Exploration

## A. Model Words

In [77]:
# Most Important Words for Each Topic
topic_words = model.topic_word_
print(topic_words.shape)

top_n = 6
topic_names_temp = []
for ii, dist in enumerate(topic_words):
    top_n_words = np.array(feature_names)[np.argsort(dist)][:-(top_n+1):-1]
    topic_names_temp.append('Topic ' + str(ii) + ': ' + ' - '.join(top_n_words.flatten()))
    print('Topic {}:  {}'.format(ii, ' - '.join(topic_names_temp.flatten())))
    

(56L, 363136L)
Topic 0:  open source - vice president - law school - best practice - big data - web site
Topic 1:  running time - information visit - fly fishing - game throne - happy hour - book club
Topic 2:  facebook group - make money - youll learn - online business - resource mentioned - email list
Topic 3:  fantasy football - radio network - day trial - get first - blue apron - first order
Topic 4:  research practice - bible church - god word - solar system - truth god - bible teaching
Topic 5:  false false - style definition - definition tablemsonormaltable - msostylename table - msostylenoshow yes - tablemsonormaltable msostylename
Topic 6:  google play - content marketing - internet marketing - key point - tunein radio - domain name
Topic 7:  help show - dot com - walking dead - please consider - gmail dot - theme song
Topic 8:  please leave - de la - help people - show please - youre going - dont miss
Topic 9:  week show - week talk - show week - live show - new show - joinin

## B. Show Topics

In [78]:
model.doc_topic_.shape

(11738L, 56L)

In [125]:
# Best Fitting Episodes for Each Topic
show_topics = model.doc_topic_
print(show_topics.shape)

# Get closest topic for each show
show_closest_topic = np.array(topic_names_temp)[show_topics.argmax(1)]
# Get probability of closest topic for each show
show_closest_topic_prob = show_topics.max(1)

# Create data frame with the above two vectors
show_closest_topic = pd.DataFrame({'usub': show_closest_topic,
                                   'usub_prob': show_closest_topic_prob})
# Add in the full show table
show_closest_topic = pd.concat([show_closest_topic, shows], axis=1)
# Extract the ID from the usub
show_closest_topic['usub_id'] = show_closest_topic['usub'].str.extract('(Topic [0-9]{1,2})').str.replace('Topic ', '')

# Reorder and rename columns
show_closest_topic = show_closest_topic[['usub_id', 'usub', 'usub_prob', 'podcast_name', 'subgenre', 'show_desc']]
show_closest_topic = show_closest_topic.rename(columns={'subgenre': 'isub'})


(11738L, 56L)


In [126]:
show_closest_topic.head()

,usub_id,usub,usub_prob,podcast_name,isub,show_desc
0,25,Topic 25: contact u - major spoiler - tell fri...,0.386675,"! Football Ringtones, Text Tones, Mail Alerts ...",College & High School,Get FREE RINGTONES when you subscribe! Footbal...
1,49,Topic 49: apple watch - app store - facebook g...,0.394105,! World's Funniest Ringtones for iPhone & iPad...,Gadgets,Get FREE RINGTONES when you subscribe to The W...
2,25,Topic 25: contact u - major spoiler - tell fri...,0.243341,! iPhone Ringtones,Video Games,Get FREE RINGTONES when you subscribe! The Wor...
3,47,Topic 47: radio show - talk radio - attention ...,0.331966,""" Talk to Tony - Radio Show "" Licensed Psychol...",Social Sciences,Dr. Tony Ferretti offers a psychological persp...
4,36,Topic 36: guest today - share tip - today talk...,0.658082,"""A Tale of Two Cities"" Audiobook (Audio book)",Literature,"""A Tale of Two Cities"" was first published in ..."


In [127]:
# Export to CSV
show_closest_topic.to_csv('../output/' + scenario + '_shows_and_closest_topics.csv', index=False)

# V. Evaluation

In [ ]:
# Load category mapping
cat_map = pd.read_csv('../raw/cat_maps/cat_map_' + scenario + '.csv')

In [130]:
# Load pickled model
model = joblib.load('../interim/trained_models/lda/lda_' + scenario + samp + '.pkl')